In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/data.csv')

In [ ]:
train.head()

## As  these kernel is much focussed on the statistical analysis , but what is a kernel on a kaggle without EDA , lets get some EDA done .

In [ ]:
train.columns 

In [ ]:
train.isnull().sum()

## The dataset is clean , since there are no missing values , lets look for some outliars 

In [ ]:
train.describe()

## its difficult to say whether there are some outliars in this dataset since the context of the columns are not so clear . We will explore this more after a while 

In [ ]:
## since the id and 'Unnamed: 32' are useless we will simply drop them.
train = train.drop(['id' , 'Unnamed: 32'] , axis = 1)

In [ ]:
train.head()

In [ ]:
import matplotlib.pyplot as plt 
plt.figure(figsize = (8,4))
train.diagnosis.value_counts().plot(kind = 'bar')

## so from the plot it is evident that it is not highly imbalanced it is imbalanced but not so much 


## Lets get more statisticaland reinforce our EDA 
## first deal with the outlier problem , just by looking the summary of this dataset it is hard to tell that theree are any exception or not 
## histogram comes to our rescue .

* How many times each value appears in dataset. This description is called the distribution of variable
* Most common way to represent distribution of varible is histogram that is graph which shows frequency of each value.
* Frequency = number of times each value appears
* Example: [1,1,1,1,2,2,2]. Frequency of 1 is four and frequency of 2 is three.

In [ ]:
m = plt.hist(train[train['diagnosis'] == "M"].radius_mean , bins = 30 ,fc = (1,0,0,0.5) , label = 'Malignm')
b = plt.hist(train[train['diagnosis'] == "B"].radius_mean , bins = 30 ,fc = (0,1,0,0.5), label = 'Belignm')
## fc is the facecolor which will be used to change (R , G , B , A) values , it doesnot affect the 
## transparncy of the histogram 
plt.legend()
plt.xlabel('Radius mean values')
plt.ylabel('Frequency')
plt.title('Histogram of radus mean for being Malignm and Belignm')
plt.grid('True')
plt.show()


In [ ]:
print(type(m))
print(m)
print(m.index)

In [ ]:
## lets write a function to build the histogram for various features of the dataset
## result : this is the wrong function
def fun_hist(feature_list):
    for i in feature_list:
        m = plt.hist(train[train['diagnosis'] == "M"][i] , bins = 30 , label = 'Malignm')
        b = plt.hist(train[train['diagnosis'] == "B"][i] , bins = 30 ,  label = 'Belignm')
        plt.xlabel(i)
        plt.ylabel('Frequency')
        plt.title('Histogram of features' )

In [ ]:
features = ['texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean']

In [ ]:
fun_hist(features)

## this doesn't turned out what we expected
## lets try to build the subplot of different histograms https://www.kaggle.com/residentmario/subplots


In [ ]:
fig , axarr = plt.subplots(2,1 , figsize = (12,8))

train[train['diagnosis'] == 'M']['radius_mean'].plot.hist(ax = axarr[0] , title  ='Hist' , fc = (1,0,0,.5))
train[train['diagnosis'] == 'B']['radius_mean'].plot.hist(ax = axarr[0] , title  ='Hist' , fc=(0,1,0,.5))
plt.title('hist')

train[train['diagnosis'] == 'M']['texture_mean'].plot.hist(ax = axarr[1] , title  ='Hist', fc = (1,0,0,.5) )
train[train['diagnosis'] == 'B']['texture_mean'].plot.hist(ax = axarr[1] , title  ='Hist', fc =(0,1,0,.5) )
plt.title('rect')

In [ ]:
features =  ['texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean']

In [ ]:
## lets try to write the function for the same task 
## wrong function 
nrows = len(features)
fig , axarr = plt.subplots(nrows , 1 , figsize=(12,8))
for i in features:     ## seems like i will have to use double for loop 
     for t in range(nrows):        ## which will be inefficient , but that not my primary concern now 
            train[train['diagnosis'] == "M"][i].plot.hist(ax = axarr[t] , fc = (1,0,0,.5))
            train[train['diagnosis'] == "B"][i].plot.hist(ax = axarr[t] , fc =(0,1,0,.5))

## this is'nt something which i expected , we will try to get it right ......
## we will try to solve this issue in the other part of this notebook 

## OK , now lets deal with the outliers 

* While looking histogram as yok can see there are rare values in bening distribution (green in graph)
* There values can be errors or rare events.
* These errors and rare events can be called outliers.
* Calculating outliers:
* first we need to calculate first quartile (Q1)(25%)
* then find IQR(inter quartile range) = Q3-Q1
* finally compute Q1 - 1.5IQR and Q3 + 1.5IQR
*Anything outside this range is an outlier
*lets write the code for bening tumor distribution for feature radius mean

In [ ]:
## since we donot have any clue about the dataset , we will move forward by statistics 
data_bening = train[train["diagnosis"] == "B"]
data_malignant = train[train["diagnosis"] == "M"]
desc = data_bening.radius_mean.describe()
Q1 = desc[4]
Q3 = desc[6]
IQR = Q3-Q1
lower_bound = Q1 - 1.5*IQR
upper_bound = Q3 + 1.5*IQR
print("Anything outside this range is an outlier: (", lower_bound ,",", upper_bound,")")
data_bening[data_bening.radius_mean < lower_bound].radius_mean
print("Outliers: ",data_bening[(data_bening.radius_mean < lower_bound) | (data_bening.radius_mean > upper_bound)].radius_mean.values)

## we can find outliers even from the boxplot


In [ ]:
melted_data = pd.melt(train,id_vars = "diagnosis",value_vars = ['radius_mean', 'texture_mean'])
melted_data.head()

In [ ]:
import seaborn as sns
plt.figure(figsize = (15,10))
sns.boxplot(x = "variable", y = "value", hue="diagnosis",data= melted_data)

## we can see the same three outlier values in the radius _mean graph 

## lets see the summary of our dataset
## Summary 
* Mean
* Variance: spread of distribution
* Standart deviation square root of variance
* Lets look at summary statistics of bening tumor radiance mean

In [ ]:
print("mean: ",data_bening.radius_mean.mean())
print("variance: ",data_bening.radius_mean.var())
print("standart deviation (std): ",data_bening.radius_mean.std())
print("describe method: ",data_bening.radius_mean.describe())

## CDF 
* Cumulative distribution function is the probability that the variable takes a value less than or equal to x. P(X <= x)
* Lets explain in cdf graph of bening radiues mean
* in graph, what is P(12 < X)? The answer is 0.5. The probability that the variable takes a values less than or equal to 12(radius mean) is 0.5.
* You can plot cdf with two different method

In [ ]:
plt.hist(data_bening.radius_mean,bins=50,fc=(0,1,0,0.5),label='Bening',normed = True,cumulative = True)

In [ ]:
## important to remember wht is the significance of normed , it scales the Y axis from 0 to 1 
plt.hist(data_bening.radius_mean,bins=50,fc=(0,1,0,0.5),label='Bening',normed = False,cumulative = True)

In [ ]:
plt.hist(data_bening.radius_mean,bins=50,fc=(0,1,0,0.5),label='Bening',normed = True,cumulative = True)
sorted_data = np.sort(data_bening.radius_mean)
y = np.arange(len(sorted_data))/float(len(sorted_data)-1)
print(y)
plt.plot(sorted_data,y,color='red')
plt.title('CDF of bening tumor radius mean')
plt.show()


In [ ]:
# lets analyse the relationshio between radius_mean and area_mean
plt.figure(figsize=(6,6))
plt.scatter(x = 'radius_mean' , y = 'area_mean' , data = train)
plt.xlabel('radius_mean')
plt.ylabel('area_mean')

## looks like a positive corelation between radius_mean and area_mean
## check for normality 


In [ ]:
from scipy.stats import norm
from scipy import stats
sns.distplot(train['radius_mean'] , fit = norm);
fig = plt.figure()

r = stats.probplot(train['radius_mean'] , plot = plt)

In [ ]:
#positive skewdness
# log transformation
import numpy as np
log_radius_mean = np.log(train['radius_mean'])

In [ ]:
sns.distplot(log_radius , fit = norm)
fig = plt.figure()
t = stats.probplot(log_radius_mean , plot = plt)

## looks like we fix the skewdness and make it normal 
## next up area_mean


In [ ]:
sns.distplot(train['area_mean'] , fit = norm)
fig = plt.figure()
c = stats.probplot(train['area_mean'] , plot = plt) 

## again the skewedness lets appply the log transdformation 
## there are similarites between distplot and histogram 

In [ ]:
log_area_mean = np.log(train['area_mean'])
sns.distplot(log_area_mean , fit  = norm)
fig = plt.figure()
x = stats.probplot(log_area_mean , plot = plt)

## yes our trick works again ,ok now get some real shit done 
## lets build the joint plot for area_mean and radius_mean and then compare it with thier log transformation 

In [ ]:
plt.figure(figsize = (8,6))
sns.jointplot(train.radius_mean , train.area_mean , kind = 'regg')
plt.show()

In [ ]:
## now build for the log transformation 
plt.figure(figsize = (8,6))
sns.jointplot(log_radius_mean , log_area_mean , kind = 'regg')

## solving the problem of the normality automatically solves the problem of homoscedanticity 
### so we have a similar thing as pairplot in seaborn which is pairgrid


In [ ]:
sns.set(style = "white")
df = train.loc[:,["radius_mean","area_mean","fractal_dimension_se"]]
g = sns.PairGrid(df,diag_sharey = False,)
g.map(plt.scatter)
plt.show()